# This Notebook is for exporting a cleaned QONNX model to be used in FINN flow

## First step is:


1.   Clone the main repository for the network
2.   install needed libraries




In [2]:
!unrar x EA-lpyolo.rar /content/

fatal: destination path 'quantized-yolov5' already exists and is not an empty directory.


In [1]:
!pip install torch torchvision numpy matplotlib
!pip install brevitas
!pip install onnx
!pip install onnxoptimizer
!pip install qonnx

  Using cached onnx-1.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
Using cached onnx-1.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.1 kB 7.8 MB/s eta 0:00:00
  Using cached qonnx-0.4.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached clize-5.0.2-py2.py3-none-any.whl.metadata (4.3 kB)
INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4

## The Second Step is:
instantiating the model

In [1]:
%cd quantized-yolov5/models/
from yolo import Model
import onnx
import torch
import os

/content/quantized-yolov5/models


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
x = Model()

Reversing anchor order


/usr/local/lib/python3.11/dist-packages/torch/_tensor.py:1624: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1935.)
  return super().rename(names)


### Importing important libraries

In [3]:
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
build_dir = '/content/quantized-yolov5/'

### Loading the model with trained weigths

In [4]:
import os
import torch

# Make sure the model is on CPU before loading a pretrained state_dict
x = x.cpu()

# Load pretrained weights
trained_state_dict = torch.load(build_dir + "/best.pt")
print(trained_state_dict.keys())
x.load_state_dict(trained_state_dict, strict=False)

odict_keys(['model.0.conv.weight', 'model.0.bn.weight', 'model.0.bn.bias', 'model.0.bn.running_mean', 'model.0.bn.running_var', 'model.0.bn.num_batches_tracked', 'model.0.act.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value', 'model.2.conv.weight', 'model.2.bn.weight', 'model.2.bn.bias', 'model.2.bn.running_mean', 'model.2.bn.running_var', 'model.2.bn.num_batches_tracked', 'model.2.act.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value', 'model.4.conv.weight', 'model.4.bn.weight', 'model.4.bn.bias', 'model.4.bn.running_mean', 'model.4.bn.running_var', 'model.4.bn.num_batches_tracked', 'model.4.act.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value', 'model.6.conv.weight', 'model.6.bn.weight', 'model.6.bn.bias', 'model.6.bn.running_mean', 'model.6.bn.running_var', 'model.6.bn.num_batches_tracked', 'model.6.act.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value', 'model.8.conv.weight', 'model.8.bn.weight', 

<All keys matched successfully>

### Exporting the model
Be sure to choose the correct size for the input tensor (1 x 3 x 416 x 416) in the order of NCHW

1.   N: number of tensors per batch
2.   C: number of channels
3.   H,W: Height and Width of the tensor





In [5]:
import onnx
import torch
export_onnx_path = build_dir + "/yolo_export.onnx"
export_qonnx(x, torch.randn(1, 3, 416, 416), export_onnx_path)
qonnx_cleanup(export_onnx_path, out_file=export_onnx_path)